# Label extraction
Leer los files crudos de geo_voice data

Encontrar si un user es epidemico u expuesto en los meses previos al dataset donde se esta analizando. Estas serian las variables que querriamos predecir.

La idea es crear la lista que cuenta antennas utilizadas para cada user en el dataset outputeado por simple_format diariamente y agrupar esta lista (con repeticiones) en un agregado mensual (para luego determinar home_antenna etc.)

Tambien usando info de las antenas (epidemicas o no). Podriamos ver si un user "vive" o "paso" por la zona epidemica.

## definiciones utilizadas
* timestamp arranca en 0 segundos para 01/01/2012 00:00am
* el analisis es solo para cdrs con usuarios de la TelCo. Los users de la TelCo van todos en la primer columna (independientemente del nombre de la columna..) entonces si la llamada es entrante quiere decir que el user de la telco esta recibiendo un llamado en esa antennaID. si la llamada es saliente es al reves. (Podria pasar que sea comunicaciones inter-Telco y ahi habria dos records uno saliente y otro entrante pero con los mismos 2 userIDs).
* La Direction, viene dada relativa al user de la primer columna (incoming si entra y viceversa si es outgoing).

### el dataset de simple_format vendria con este header
 {'USER':np.int32 \\\  'OTHER_USER':np.uint32 \\\ 'Direction':np.object \\\  'TimeStamp':np.uint32  \\\ 'Duration':np.uint16 \\\  'AntennaID':np.uint16}

por lo cual seria conveniente asignar dtypes para cargar mas rapido en memoria

In [1]:
%autosave 180

Autosaving every 180 seconds


In [2]:
import pandas as pd; 
import numpy as np; 
import os;
import random;
pd.set_option('display.max_rows', 300)
from matplotlib import pyplot as plt
%matplotlib inline
import time
np.random.seed(2016)

In [3]:
!!ls /grandata/simple_format/

['SimpleFormat',
 'simple_format.sh',
 'simple_format_201401.txt.gz',
 'simple_format_201402.txt.gz',
 'simple_format_201403.txt.gz',
 'simple_format_201404.txt.gz',
 'simple_format_201405.txt.gz',
 'simple_format_201406.txt.gz',
 'simple_format_201407.txt.gz',
 'simple_format_201408.txt.gz',
 'simple_format_201409.txt.gz',
 'simple_format_201410.txt.gz',
 'simple_format_201411.txt.gz',
 'simple_format_201412.txt.gz',
 'simple_format_201501.txt.gz',
 'simple_format_201501_head.txt.gz',
 'simple_format_201502.txt.gz',
 'simple_format_201503.txt.gz',
 'simple_format_201504.txt.gz',
 'simple_format_201505.txt.gz',
 'simple_format_201506.txt.gz',
 'simple_format_201507.txt.gz',
 'simple_format_201508.txt.gz',
 'simple_format_201509.txt.gz',
 'surrogated_antennas.txt.gz',
 'surrogated_antennas_copy.txt.gz',
 'surrogated_nodes.txt.gz']

In [5]:
!zcat /grandata/simple_format/simple_format_201404.txt.gz | head 

USER OTHER_USER DIRECTION TIMESTAMP DURATION ANTENNA_ID


In [6]:
#ejemplo  "/grandata/simple_format/01-02012014.txt.gz"
#seteamos el lugar de trabajo
rootdir="/grandata/simple_format/"
os.chdir(rootdir)
year = "2014"; 
month_start= "1";
if int(month_start )<10:
    month_start="0"+month_start
    
day_start = "01"
day_end = str(int(day_start)+1)
if int(day_end )<10:
    day_end="0"+ day_end

input_file= rootdir +"simple_format_{y}{ms}.txt.gz"\
                .format(y=year,ms=month_start,
                       ds=day_start,de=day_end)
    
def get_output_file(night_filter=False,week_end=False):
    output = "/home/juan/mobility-study/past_labels"
    if week_end == True:
        output = output + "_wkend"
    if night_filter == True:
        output = output + "_ngtfilter"
    return output + ".txt.gz"

output_file = get_output_file()

In [7]:
#para enriquecer el dataset de CDRs con info acerca de la zona epidemica etc.
antennas= pd.read_csv('/home/juan/mobility-study/antennas_mexico.csv',sep = "|",header=0,index_col=0)
antennas.index.rename("ANTENNA_ID",inplace=True)

In [15]:
user_hashes_file= "/home/juan/mobility-study/output_sum_links.txt.gz"
!zcat $user_hashes_file | head

LineKeyOrigin|CallsWeekDaylight|CallsWeekDaylight_EPI|CallsWeekNight|CallsWeekNight_EPI|CallsWeekend|CallsWeekend_EPI|TimeWeekDaylight|TimeWeekDaylight_EPI|TimeWeekNight|TimeWeekNight_EPI|TimeWeekend|TimeWeekend_EPI|TOTAL_USERS|EPI_USERS|EXP_USERS
000004F96010462FA7A71C7784E27476|0|0|0|0|0|1|0|0|0|0|0|34|1|0|1
000007EEE7895E17937CFF86379C8E17|0|2|0|0|0|0|0|144|0|0|0|0|1|1|1
00000837B868F926F67BD440A0845C1D|1|0|2|4|3|0|47|0|74|86|109|0|3|0|2
00000C5D3BFBEBD919F6F68386C3DD5E|4|0|0|0|0|0|281|0|0|0|0|0|1|0|1
0000129F4191FFD0963774D2307C7510|2|0|0|0|0|0|66|0|0|0|0|0|2|0|2
000017368A8964F8C71A9F3A1EAF6160|7|4|0|0|1|0|960|630|0|0|58|0|4|1|4
00001ADC8C5705AB4FD8197B5024C1AA|7|1|5|5|4|1|744|29|207|335|218|141|4|2|4
00001B448AE41076EB2C58ABEAF0E0C8|15|0|12|1|10|1|1319|0|705|31|459|29|2|1|2
00001C544BEBBB9109CE3552B4CDC490|0|0|1|0|0|0|0|0|36|0|0|0|1|0|1

gzip: stdout: Broken pipe


In [8]:
#levanto la tabla de users hash_map.
user_hashes = pd.read_csv("/home/juan/mobility-study/output_sum_links.txt.gz",
                                  sep="|",
                                  header=0,
        dtype = {'LineKeyOrigin':np.object_,'CallsWeekDaylight':np.uint16,'CallsWeekDaylight_EPI':np.uint16,'CallsWeekNight':np.uint16,
            'CallsWeekNight_EPI':np.uint16,'CallsWeekend':np.uint16,'CallsWeekend_EPI':np.uint16, 'TimeWeekDaylight':np.uint16,
                'TimeWeekDaylight_EPI':np.uint16,'TimeWeekNight':np.uint16,'TimeWeekNight_EPI':np.uint16,'TimeWeekEnd':np.uint16
                ,'TimeWeekEnd_EPI':np.uint16,'TOTAL_USERS':np.uint16,'EPI_USERS':np.uint16,'EXP_USERS':np.uint16},
                          usecols = ['LineKeyOrigin'],
                )

In [9]:
week_end = False
night_filter = False

output_file = get_output_file()
sample_file = output_file.replace(".txt.gz","_sample.txt.gz")
#para el manejo de las distintas salidas del programa

# Corremos test sobre un sample only

In [106]:
!zcat $input_file | wc -l

428361977


In [38]:
#leo algunas primeras lineas
sample = pd.read_csv(
                input_file,
                engine = 'c',
#                chunksize = 10**7,
    #            iterator =True,
                nrows = 3*10**7,
                sep = ' ',
                header = 0,
                index_col=None,
                usecols = ['USER','ANTENNA_ID'],
                dtype = {'TIMESTAMP':np.uint32,'ANTENNA_ID':np.uint16,'USER':np.object_,'OTHER_USER':np.object_}
                )

In [39]:
len(np.unique(user_hashes.values)),user_hashes.shape,sample.shape

(2449242, (2449242, 1), (30000000, 4))

In [75]:
antennas.head()

,LATITUDE,LONGITUDE,STATE,EPIDEMIC
ANTENNA_ID,,,,
2797,14.671,-92.372,Chiapas,1
3060,14.682,-92.155,Chiapas,1
3344,14.721,-92.424,Chiapas,1
3487,14.778,-92.179,Chiapas,1
1924,14.786,-92.366,Chiapas,1


In [97]:
sample = sample[sample['USER'].isin(user_hashes.loc[:,'LineKeyOrigin'].values)]
# Obs. no es neceario filtrar por usuarios que tengan un rango de llamados minimos/maximos mensuales pues eso ya 
# estuvo previamente hecho en el filtrado anterior de sum_links y simple_format

grouped = sample.groupby(['USER', 'ANTENNA_ID'])['ANTENNA_ID'].agg({'count': np.size})
grouped.reset_index(inplace=True,drop=False)

#reordeno dentro de c/ USER por el count del antenna, esto me sirve para despues ordenar las antenas por uso
grouped.sort_values(by=['USER','count'],ascending=False,inplace=True)
grouped = pd.merge(grouped,antennas['EPIDEMIC'].reset_index(), on='ANTENNA_ID',how = 'left')

##enriquezco la muestra con datos epidemicos
#primero agrego a cada antenna del df el dato de si es epidemica
#despues agrupo por el USER y me fijo solo la columna epidemica en c/grupo
#finalmente sumo en c/ grupo y tomo la parte superior 
#entera de esa division con el largo del grupo. Si uso al menos una antena epidemica entonces esta expuesto(==1) Si no,
# da 0 pues no estuvo expuesto.        
##enriquezco la muestra con datos epidemicos

exposed_info = grouped.groupby('USER')['EPIDEMIC'].\
    agg({'EXPOSED' : lambda x: int( np.ceil(np.sum(x)*1.0/np.size(x)) )})

#actualizo la tabla grouped
grouped = grouped.join(exposed_info['EXPOSED'],on="USER",how="left")

del exposed_info

output_table = grouped.drop_duplicates(subset = 'USER', keep='first').copy()
output_table.drop('count',axis=1,inplace=True)
output_table = output_table.set_index('USER')
# y aca vamos al write

# Main algorithm 

## Ojo con los s_f corruptos
el 201404

In [10]:
!du -ha /grandata/simple_format/simple_format_* 

7.5G	/grandata/simple_format/simple_format_201401.txt.gz
6.8G	/grandata/simple_format/simple_format_201402.txt.gz
7.4G	/grandata/simple_format/simple_format_201403.txt.gz
4.0K	/grandata/simple_format/simple_format_201404.txt.gz
7.9G	/grandata/simple_format/simple_format_201405.txt.gz
7.5G	/grandata/simple_format/simple_format_201406.txt.gz
7.7G	/grandata/simple_format/simple_format_201407.txt.gz
7.8G	/grandata/simple_format/simple_format_201408.txt.gz
6.1G	/grandata/simple_format/simple_format_201409.txt.gz
6.1G	/grandata/simple_format/simple_format_201410.txt.gz
6.3G	/grandata/simple_format/simple_format_201411.txt.gz
7.0G	/grandata/simple_format/simple_format_201412.txt.gz
6.6G	/grandata/simple_format/simple_format_201501.txt.gz
4.0K	/grandata/simple_format/simple_format_201501_head.txt.gz
6.5G	/grandata/simple_format/simple_format_201502.txt.gz
7.7G	/grandata/simple_format/simple_format_201503.txt.gz
7.2G	/grandata/simple_format/simple_format_201504.txt.gz
19G	/grandata/simple_forma

In [11]:
!zcat /grandata/simple_format/simple_format_201404.txt.gz | wc -l

1


In [32]:
!du -ha /grandata/simple_format/simple_format_201404.txt.gz 

4.0K	/grandata/simple_format/simple_format_201404.txt.gz


In [31]:
!zcat /grandata/simple_format/simple_format_201404.txt.gz | head

USER OTHER_USER DIRECTION TIMESTAMP DURATION ANTENNA_ID


### Just checking..
Vemos consistencia de archivos 

In [34]:
for month in months:
    input_file= rootdir +"simple_format_{m}.txt.gz"\
            .format(m=month)
    print(month)
    !zcat $input_file | head

201401
USER OTHER_USER DIRECTION TIMESTAMP DURATION ANTENNA_ID
216E0E920C0B51093276E270CC36CB72 BA694194F2CF7978466AA9602DD38F36 O 63161302 56 1
216E0E920C0B51093276E270CC36CB72 BA694194F2CF7978466AA9602DD38F36 O 63162373 88 1
216E0E920C0B51093276E270CC36CB72 BCE827A6E83B59F75031D4FCEC36B7DD I 63217965 600 2
216E0E920C0B51093276E270CC36CB72 BCE827A6E83B59F75031D4FCEC36B7DD I 63218508 63 2
216E0E920C0B51093276E270CC36CB72 BCE827A6E83B59F75031D4FCEC36B7DD I 63217516 93 2
216E0E920C0B51093276E270CC36CB72 EADDE544AB1EC4B975AE41337ACFB5B5 I 63228387 35 2
73C2C1F8233884659CC65DF58BA2D031 39ABF1E2D91107D64EE4B6719C2D6C60 O 63286717 342 3
73C2C1F8233884659CC65DF58BA2D031 995362BDBB319FB4E367EB52E2540CC7 O 63280397 33 4
73C2C1F8233884659CC65DF58BA2D031 9D0AFC1877F147C63E235CF25B42CC0B O 63307690 38 4

gzip: stdout: Broken pipe
201402
USER OTHER_USER DIRECTION TIMESTAMP DURATION ANTENNA_ID
216E0E920C0B51093276E270CC36CB72 72868AF4DE00F3FA16B151849D0243F1 I 65891075 150 1
216E0E920C0B51093276E270

In [6]:
months14= ["2014"+ "0"+str(month) if int(month )<10 else "2014"+str(month) for month in range(1,13)]
months15= ["2015"+ "0"+str(month) if int(month )<10 else "2015"+str(month) for month in range(1,5)]
months = months14  + months15

In [142]:

## script para extraer los labels de los simple_formats de 2014
#cada dataset tiene aprox 450M de datos, de los cuales aprox 45M son de los Telco users que nos intersan
# como hay 16 meses a analizar ---> tengo 720M registros que mirar, luego hago 15 pasadas filtrados por el user_hash

#ver el tiempo que tarda
start_time = time.time()

#el chunk basicamente va leyendo el file de a 'chunksize' cantidad de filas
#subgroup = pd.DataFrame()

#hay 12 meses del cual extraer atributos:
months14= ["2014"+ "0"+str(month) if int(month )<10 else "2014"+str(month) for month in range(1,13)]
months15= ["2015"+ "0"+str(month) if int(month )<10 else "2015"+str(month) for month in range(1,5)]
months = months14  + months15

#aca seteamos como vamos a partir la tabla segun el nro correspondiente a c/hash y tomando modulo
passes = 15
#creo los grupos que despues van a filtrar c/chunk de la tabla para hacer varias pasadas
for group in range(passes):
    print("\n working group number {it} of {pas}, time elapsed is {t} \n".\
          format(it=group,pas=passes, t=(time.time()-start_time)))

    #itero sobre los meses
    subgroup = pd.DataFrame()
    for month in months:
        if month == '201404':
            continue
            
        input_file= rootdir +"simple_format_{m}.txt.gz"\
            .format(y=year,m=month)

        table = pd.read_csv(
                input_file,
                engine = 'c',
                chunksize = 11*10**7,
    #            iterator =True,
                sep = ' ',
                header = 0,
                index_col=None,
                usecols = ['USER','ANTENNA_ID'],
                error_bad_lines= False,
                warn_bad_lines= True,
                dtype = {'ANTENNA_ID':np.uint16,'USER':np.object_}
                )

        #cuando entramos a este loop, table tiene tantos 'chunks' como el valor entero de la cantidad de lineas en el file
        #dividido el tamanyo del chunksize

        numb=0;
        for chunk in table:
                numb+=1
                #aca filtro por la cantidad 
                chunk = chunk[chunk['USER'].isin(user_hashes.loc[:,'LineKeyOrigin'].values)]
                #a cada chunk filtro por todos los USERs por el hash del string (el hash es un int) y despue filtro modulo passes
                #y trabajo sobre la tabla subgroup nada mas que ahora tiene menos usuarios
                subgroup = subgroup.append(chunk[chunk['USER'].apply(lambda x: hash(x) % passes == group )])
    
        print("Finished parsing month {ms}-{y}, time elapsed is {t} ".format(ms=month,y=year,t=(time.time()-start_time)))
        
    #entonces la idea es que yo ahora solo voy a trabajar, dentro de esta tabla filtrada y para todos los meses juntos
    
    #paso de segundos a horas
    #notar que TIMESTAMP arranca en 0 segundos para domingo 01/01/2012 00:00am 
    #con lo cual domingo es el dia 0, lunes el 1, asi..
    print('finished reading months for group {it} of {pas}: time elapsed is {t} \n'.\
    format(it=group,pas=passes, t=(time.time()-start_time)) )
    
    #filtro usuarios con pocos o demasiados llamados en general menos de 25 y mas de 2000 en 5 meses  
    grouped = subgroup.groupby(['USER', 'ANTENNA_ID'])['ANTENNA_ID'].agg({'count': np.size})
    
    del subgroup
    grouped.reset_index(inplace=True,drop=False)   
   
    #reordeno dentro de c/ USER por el count del antenna, esto me sirve para despues ordenar las antenas por uso
    grouped.sort_values(by=['USER','count'],ascending=False,inplace=True)
    grouped = pd.merge(grouped,antennas['EPIDEMIC'].reset_index(), on='ANTENNA_ID',how = 'left')

    ##enriquezco la muestra con datos epidemicos
    #primero agrego a cada antenna del df el dato de si es epidemica
    #despues agrupo por el USER y me fijo solo la columna epidemica en c/grupo
    #finalmente sumo en c/ grupo y tomo la parte superior 
    #entera de esa division con el largo del grupo. Si uso al menos una antena epidemica entonces esta expuesto(==1) Si no,
    # da 0 pues no estuvo expuesto.        
    ##enriquezco la muestra con datos epidemicos

    exposed_info = grouped.groupby('USER')['EPIDEMIC'].\
        agg({'EXPOSED' : lambda x: int( np.ceil(np.sum(x)*1.0/np.size(x)) )})

    #actualizo la tabla grouped
    grouped = grouped.join(exposed_info['EXPOSED'],on="USER",how="left")
    del exposed_info

    #creo la tabla filtrada solo por users, que es la que voy a terminar guardando (hay tantos rows como users)
    output_table = grouped.drop_duplicates(subset = 'USER', keep='first')
    del grouped

    #re indexo
    output_table.set_index('USER',inplace=True,drop=False)
    
    #agrupo ahora la tabla por USER para hacer todos los calculos en los grupos
    output_table.drop('count',axis=1,inplace=True)
    output_table = output_table.set_index('USER')
    #paso a ints todas las columnas (el hash ya esta de index)
    output_table = output_table.astype(int,copy=False)
    
    #aca termino guardando (en forma de append) el output final pero solo para esos usuarios % pass ==group

    #el primer write (first group==0) va con header
    if group ==0 :
        output_table.to_csv(output_file, index = True,
                   header = True,sep = "|", 
                            compression = "gzip", mode='w')
    else: 
        output_table.to_csv(output_file, index = True,
                   header = False,sep = "|", 
                    compression = "gzip", mode='a')

    print("Finished group {gr} of {ps}, time elapsed is {t}\n ".format(gr=group,ps=passes,t=(time.time()-start_time)))

print("total running time of script is %d " % time.time() - start_time)


 working group number 0 of 15, time elapsed is 0.00229692459106 



CParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

## cambiamos el working dir y el input_file (output.txt.gz)